# Tabla de contenidos
* [Tabla de contenidos](#Tabla-de-contenidos)
* [<font color='blue'> $k$-VECINOS MÁS CERCANOS <br> </font>](#<font-color='blue'>-$k$-VECINOS-MÁS-CERCANOS-<br>-</font>)
	* [<font color='blue'> Introducción </font>](#<font-color='blue'>-Introducción-</font>)
	* [<font color='blue'> Algoritmo k-Nearest Neighbours (k-NN) </font>](#<font-color='blue'>-Algoritmo-k-Nearest-Neighbours-%28k-NN%29-</font>)
	* [<font color='blue'> Pasos del Algoritmo </font>](#<font-color='blue'>-Pasos-del-Algoritmo-</font>)
	* [<font color='blue'> Ejemplo de clasificación con el conjunto de datos Iris </font>](#<font-color='blue'>-Ejemplo-de-clasificación-con-el-conjunto-de-datos-Iris-</font>)
		* [Importar las bibliotecas](#Importar-las-bibliotecas)
		* [Importar el conjunto de datos](#Importar-el-conjunto-de-datos)
		* [Escalar los datos](#Escalar-los-datos)
		* [Generar subconjuntos de entrenamiento y test](#Generar-subconjuntos-de-entrenamiento-y-test)
		* [Construir el modelo $k$-NN](#Construir-el-modelo-$k$-NN)
		* [Evaluar el modelo](#Evaluar-el-modelo)
		* [Elegir el mejor valor de $k$](#Elegir-el-mejor-valor-de-$k$)
		* [Clasificar nuevos ejemplos](#Clasificar-nuevos-ejemplos)
	* [<font color='blue'> Conclusiones </font>](#<font-color='blue'>-Conclusiones-</font>)


# <font color='blue'> $k$-VECINOS MÁS CERCANOS <br> </font>

## <font color='blue'> Introducción </font>

* Se basa en el almacenamiento de ejemplos y, cuando se quiere clasificar un nuevo objeto, se extraen los objetos más parecidos y se usa su clasificación para clasificar al nuevo objeto.
* Su mayor diferencia con otros algoritmos se basa en la trivialidad del proceso de aprendizaje.

## <font color='blue'> Algoritmo k-Nearest Neighbours (k-NN) </font>

* Es sencillo y fácil de implementar
* Robusto frente al ruido
* Sus funciones de proximidad (funciones que deciden la clasificación):
    * Retomamos las funciones de distancia usadas para clustering
    * Los atributos numéricos pueden sufrir un proceso de normalización para evitar que atributos con valores altos tengan más peso
    * Admite funciones de proximidad que asignen peso a los atributos, lo que permite eliminar atributos irrelevantes, asignar más peso a los ejemplos más cercanos, ...
* El resultado de la clasificación será:
    * la clase más frecuente de los k-vecinos (clases discretas)
    * la media de las clasificaciones (clases continuas)

## <font color='blue'> Pasos del Algoritmo </font>

1. Calcula la distancia entre la observación a clasificar y el resto de ejemplos del conjunto de entrenamiento.
1. Se seleccionan los $k$ registros con menor distancia a la observación.
1. Si todos pertenecen a la misma clase, dicha observación es clasificada en esa clase.
1. Si no todos pertenecen a la misma clase, dicha observación se clasifica a la clase más votada, la más frecuente.

Para aplicar este algoritmo se puede consultar 

[sklearn.neighbors.KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)

de dónde han sido extraídos los ejemplos que se muestran a lo largo de este cuaderno.

A continuación se muestra un ejemplo sencillo, considerando el valor de $k$=3 y siendo el conjunto de entrenamiento X y la salida y:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
X = [[0], [1], [2], [3]]
y = [0, 0, 1, 1]
kNN = KNeighborsClassifier(n_neighbors=3)
kNN.fit(X, y) 

In [ ]:
# Se predice la salida para estos ejemplos:
print(kNN.predict([[1.1]]))
print(kNN.predict_proba([[0.9]]))

Este algoritmo también se puede usar en aprendizaje no supervisado. Fíjese en el siguiente ejemplo, en este caso _fit_ sólo usa el parámetro de entrada, no recibe la salida. 

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
X

In [ ]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

# indices[:, 1] toma el segundo vecino (el más cercano que no sea el propio punto)
indices = indices[:, 1]
distances = distances[:, 1]

print("Vecinos más cercanos (excluyendo el propio punto):")
print(indices)
print("Distancias:")
print(distances)

## <font color='blue'> Ejemplo de clasificación con el conjunto de datos Iris </font>

### Importar las bibliotecas

El primer paso es importar las bibliotecas.

In [ ]:
# Preprocesado y modelado
# ------------------------------------------------------------------------------
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, GridSearchCV 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

from sklearn.neighbors import KNeighborsClassifier  # k-NN para clasificación


# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix


# Configuración warnings
# ------------------------------------------------------------------------------
from warnings import filterwarnings
filterwarnings('ignore')

### Importar el conjunto de datos

In [ ]:
# Cargar el conjunto de datos Iris
iris = datasets.load_iris()

In [ ]:
# Variables independientes
X = iris.data[:, :]
# Variable a predecir
y = iris.target

In [ ]:
print(X)
print(y)

### Generar subconjuntos de entrenamiento y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Escalar los datos

In [ ]:
scaler = MinMaxScaler().fit(X_train)

# Transformar el conjunto de datos
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)
print(X_test)

Todos los valores de las variables están ahora escalados entre 0 y 1.

### Construir el modelo $k$-NN

In [ ]:
# Fijar el valor de la vecindad
n_neighbors = 3
 
# Crear objeto k-NN classifer
knn = KNeighborsClassifier(n_neighbors, metric='euclidean')

# Entrenar el clasificador k-NN
knn.fit(X_train, y_train)

### Evaluar el modelo

In [ ]:
# Evaluar la respuesta sobre el mismo conjunto de entrenamiento
print('Acc del clasificador k-NN sobre el conjunto de entrenamiento: {:.2f}'
     .format(knn.score(X_train, y_train)))

Confirmemos la precisión obteniendo la matriz de confusión y _report_ sobre la predicción del conjunto test:

In [ ]:
# Predecir la respuesta sobre el conjunto de prueba

# Se puede usar la función Score
print('Acc del clasificador k-NN sobre el conjunto test: {:.2f}'
     .format(knn.score(X_test, y_test)))


In [ ]:
# O bien algunas métricas como la matriz de confusión, precision, recall,...
y_pred = knn.predict(X_test)
print('Matriz de confusión: ')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Elegir el mejor valor de $k$

Vamos a representar gráficamente los resultados obtenidos con distintos valores de $k$, de manera que nos ayude a seleccionar un valor óptimo para este parámetro.

In [ ]:
k_range = range(1,10)
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
plt.figure()
plt.title('Accuracy Rate K Value')
plt.xlabel('k value')
plt.ylabel('Accuracy')
plt.scatter(k_range,scores)
plt.xticks([0,2,4,6,8,10])
plt.show()

In [ ]:
print(scores)

Observando la gráfica se puede seleccionar el mejor valor para $k$.

También se puede buscar el mejor valor para $k$ usando la conocida función GridSearchCV. Es una buena solución que permite hacer una búsqueda de los valores de los hiperparámetros en los modelos de _Machine Learning_. 

GridSearchCV está disponible en scikit-learn.

Obtenemos automáticamente el mejor valor de $k$ para este conjunto de datos:

In [ ]:
parameters = {"n_neighbors": range(1, 20)}
gridsearch = GridSearchCV(KNeighborsClassifier(), parameters)
gridsearch.fit(X_train, y_train)

In [ ]:
gridsearch.best_params_

Puede usarse para todos los parámetros deseados:


In [ ]:
parameters = {
...     "n_neighbors": range(1, 20),
...     "weights": ["uniform", "distance"],
... }

gridsearch = GridSearchCV(KNeighborsClassifier(), parameters, cv= RepeatedKFold(n_splits = 10,n_repeats=5))
gridsearch.fit(X_train, y_train)

In [ ]:
# Mejores hiperparámetros encontrados 
print("Mejores valores de los hiperparámetros:")
print(gridsearch.best_params_)

In [ ]:
# Evaluar el modelo

accuracy = gridsearch.best_score_ *100
print("Precisión del conjunto de entrenamiento: {:.2f}%".format(accuracy))

# Se aplica al conjunto test
y_pred=gridsearch.predict(X_test)
test_accuracy=accuracy_score(y_test,y_pred)*100

print("Precisión del conjunto test: {:.2f}%".format(test_accuracy))
print(classification_report(y_test, y_pred)) 

In [ ]:
# Visualizando la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm)

### Clasificar nuevos ejemplos

Cuando se alcance unos valores de las métricas que permitan aceptar el modelo como válido y que, por tanto, está listo para su uso, se llevaría a un entorno real y productivo. En esa situación el modelo se enfrentará a datos totalmente desconocidos, los datos almacenados sobre el conjunto test_real, y no se podrá evaluar la respuesta del modelo, salvo las indicaciones que puedan arrojar los expertos en la materia a partir del conocimiento del dominio del problema a resolver.

La predicción se puede hacer de dos formas:

In [ ]:
# Predice la clase:
print(gridsearch.predict([[6, 2, 4.9, 1.5]]))

In [ ]:
# Predice la probabilidad de pertenencia a cada clase
print(gridsearch.predict_proba([[6, 2, 4.9, 1.5]]))

100% de probabilidad para la clase 2.

## <font color='blue'> Conclusiones </font>

Se ha construido un modelo de clasificación para procesar y clasificar puntos de un conjunto de entrada con el algoritmo $k$-Nearest Neighbour. Como su nombre indica, se evalúan los $k$ vecinos más cercanos para clasificar nuevos puntos. Al ser utilizado como un algoritmo supervisado, se debe disponer de suficientes muestras etiquetadas para poder entrenar el modelo con buenos resultados. Este algoritmo es bastante simple y poderoso, pero puede necesitar mucha memoria y recursos de CPU para procesar un gran conjunto de datos y evaluar nuevos puntos. Esto no lo hace recomendable para conjuntos de datos muy grandes.

El algoritmo $k$-NN se usa para encontrar similitudes de documentos y reconocimiento de patrones. También se emplea para desarrollar sistemas de recomendación y para la reducción de la dimensionalidad, así como en las fases del preprocesamiento para la visión artificial, en particular, las tareas de reconocimiento facial.

<img src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png"/> 

Esta obra está bajo una Licencia Creative Commons Atribución-NoComercial-CompartirIgual 4.0 Internacional.
Para ver una copia de esta licencia, véase http://creativecommons.org/licenses/by/4.0/